In [ ]:
import numpy as np
import os
from skimage import io, transform
from skimage.measure import marching_cubes
import vtk
import trimesh
import cv2 as cv
import cumcubes
import torch


In [ ]:
def load_image(file_path):
    with Image.open(file_path) as img:
        return np.array(img)

def load_masks_to_volume(mask_folder, kidney_folder, cuda=True, scale=1.0, blur_kernel=(25,25)):
    mask_files = sorted([f for f in os.listdir(mask_folder) if f.endswith('.tif')])
    kid_files = sorted([f for f in os.listdir(kidney_folder) if f.endswith('.tif')])
    
    first_mask = io.imread(os.path.join(mask_folder, mask_files[0]))
    height, width = first_mask.shape[:2]
    new_width, new_height = (int(width * scale), int(height * scale))
    volume_shape = (len(mask_files), int(height * scale), int(width * scale))
    
    if cuda:
        volume = torch.zeros(volume_shape, dtype=torch.uint8)
        volume1 = torch.zeros(volume_shape, dtype=torch.uint8)

        for i, mask_file in enumerate(mask_files):
            img = io.imread(os.path.join(mask_folder, mask_file))

            resized = transform.resize(img, (new_height, new_width), anti_aliasing=True)

            mask = torch.from_numpy(resized)
            volume[i, :, :] = mask
            
        for i, file in enumerate(kid_files):
            img = cv.imread(os.path.join(kidney_folder, file), cv.IMREAD_GRAYSCALE)

            resized = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_AREA)

            blur = cv.GaussianBlur(resized, blur_kernel, 0)

            _, thresh = cv.threshold(blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
            thresh = torch.from_numpy(thresh)
            
            contours, _ = cv.findContours(thresh.numpy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv.contourArea, reverse=True)
            largest_contour = contours[0]
            
            mask = torch.zeros_like(thresh)
            cv.drawContours(mask.numpy(), [largest_contour], -1, (255), -1)
            
            volume1[i, :, :] = mask

        volume = volume.cuda()
        volume1 = volume1.cuda()

        return volume, volume1

    else:
        volume = np.zeros(volume_shape, dtype=np.uint8)
        volume1 = np.zeros(volume_shape, dtype=np.uint8)

        for i, mask_file in enumerate(mask_files):
            img = io.imread(os.path.join(mask_folder, mask_file))

            resized = transform.resize(img, (new_height, new_width), anti_aliasing=True)

            volume[i, :, :] = resized
            
        for i, file in enumerate(kid_files):
            img = cv.imread(os.path.join(kidney_folder, file), cv.IMREAD_GRAYSCALE)
            resized = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_AREA)

            blur = cv.GaussianBlur(resized, blur_kernel, 0)
            _,thresh = cv.threshold(blur,0,255,cv.THRESH_BINARY+ cv.THRESH_OTSU)

            contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv.contourArea, reverse=True)
            largest_contour = contours[0]

            cv.drawContours(volume1[i], [largest_contour], -1, (255), -1)

            #volume1[i, :, :] = thresh

        return volume, volume1


mask_folder = 'C:\\Users\\Bartek\\Desktop\\mgr\\train\\kidney_1_dense\\labels'
kidney_contour_dir = 'C:\\Users\\Bartek\\Desktop\\mgr\\train\\kidney_1_dense\\images'

volume, volume1 = load_masks_to_volume(mask_folder, kidney_contour_dir, cuda=False, blur_kernel=(15,15))
print("volume")

In [ ]:
def create_mesh_from_volume(volume, smoothing=True, level=0.5, iterations=10, cuda=True):
    if cuda:
        verts, faces = cumcubes.marching_cubes(volume, 0, verbose=True)
        verts = verts.cpu().numpy()
        faces = faces.cpu().numpy()
    else:
        verts, faces, _, _ = marching_cubes(volume, level=level)
    print("aaa")
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)

    if smoothing: 
        mesh = trimesh.smoothing.filter_laplacian(mesh, iterations=iterations)

    return mesh

kidney_mesh = create_mesh_from_volume(volume1, cuda=False, smoothing=True, iterations=5)
print("mesh1")
blood_mesh = create_mesh_from_volume(volume, cuda=False, smoothing=True, iterations=10)
print("mesh2")
kidney_mesh.export("kidney_reconstruction_cuda.stl")
blood_mesh.export("blood_vessels_reconstruction2_cuda.stl")
print("done")

In [ ]:
del volume1
torch.cuda.empty_cache()

In [ ]:
vertices_cu, faces_cu = cumcubes.marching_cubes(volume, 0, verbose=True)

In [ ]:
vertices = vertices_cu.cpu().numpy()
faces = faces_cu.cpu().numpy()

# Create the mesh using trimesh
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
mesh.export('output_mesh.stl')

In [ ]:
verts_o3d = o3d.utility.Vector3dVector(vertices)
faces_o3d = o3d.utility.Vector3iVector(faces)

# Create an Open3D mesh
mesh = o3d.geometry.TriangleMesh(vertices=verts_o3d, triangles=faces_o3d)

o3d.io.write_triangle_mesh('C:\\Users\\Bartek\\Desktop\\\mgr\\output_mesh1.stl', mesh)
if False:
    mesh = mesh.filter_smooth_laplacian(number_of_iterations=10)


In [ ]:
verts_o3d

In [ ]:
mesh = manifold.load_mesh('path_to_your_mesh_file.stl')

# Apply smoothing (adjust the method based on Manifold's API)
smoothed_mesh = manifold.smooth_mesh(mesh, iterations=10)

# Save the smoothed mesh
manifold.save_mesh(smoothed_mesh, 'path_to_output_file.stl')